In [1]:
import pypsa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import cartopy.crs as ccrs

In [3]:
csv_folder_name = "Nepal_Data"

fig_size = (12,6)

network = pypsa.Network()
network.import_from_csv_folder(csv_folder_name)

INFO:pypsa.io:Imported network Nepal_Data has buses, generators, lines, loads, transformers


In [4]:
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 5 iterations with error of 0.000000 in 0.399266 seconds


{'n_iter': SubNetwork  0
 snapshot     
 now         5,
 'error': SubNetwork             0
 snapshot                
 now         6.713776e-10,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [5]:
network.buses_t.p

Bus,Mahendranagar_132,Mahendranagar_33,Attariya_132,Attariya_33,Chameliya HEP,Chameliya HEP_132,Syaule_132,Syaule_33,Pahalmanpur_132,Pahalmanpur_33,...,Iwa Khola HEP,Iwa Khola HEP_132,Solu Khola HEP,Solu Khola HEP_132,Grid Connected Solar PV,Grid Connected Solar PV_132,Daraudi Khola HEP,Daraudi Khola HEP_132,Rudi Khola HEP,Rudi Khola HEP_132
snapshot,,,,,,,,,,,,,,,,,,,,,
now,0.0,-14.11,0.0,-13.77,30.0,0.0,0.0,-6.8,0.0,-6.8,...,47.0,0.0,95.0,0.0,8.55,0.0,37.0,0.0,20.0,0.0


In [9]:

import pypsa
import pandas as pd
import uuid

# Step 1: Initialize PyPSA network
n = pypsa.Network()

# Step 2: Load data from CSV files
buses = pd.read_csv("Nepal_Data/buses.csv")
loads = pd.read_csv("Nepal_Data/loads.csv")
generators = pd.read_csv("Nepal_Data/generators.csv")
lines = pd.read_csv("Nepal_Data/lines.csv")
transformers = pd.read_csv("Nepal_Data/transformers.csv")

# Step 3: Add buses
bus_index = {}  # Map bus names to indices for MATPOWER
for i, row in buses.iterrows():
    n.add("Bus", row["name"], v_nom=row["v_nom"], type=row["control"].lower())
    bus_index[row["name"]] = i + 1  # MATPOWER bus indices start at 1

# Step 4: Add loads
for _, row in loads.iterrows():
    n.add("Load", row["name"], bus=row["bus"], p_set=row["p_set"], q_set=row["q_set"])

# Step 5: Add generators
for _, row in generators.iterrows():
    n.add("Generator", row["name"], bus=row["bus"], control=row["control"], p_nom=row["p_nom"], p_set=row["p_set"])

# Step 6: Add lines
for _, row in lines.iterrows():
    n.add("Line", row["name"], bus0=row["bus0"], bus1=row["bus1"], r=row["r"], x=row["x"], b=row["b"], s_nom=100)  # Assume 100 MVA if not specified

# Step 7: Add transformers
for _, row in transformers.iterrows():
    n.add("Transformer", row["name"], bus0=row["bus0"], bus1=row["bus1"], r=row["r"], x=row["x"], s_nom=row["s_nom"])

# Step 8: Run power flow
n.pf()

# Step 9: Generate MATPOWER .m file content
matpower_content = []

# Bus data
matpower_content.append("%% bus data")
matpower_content.append("% bus_i type Pd Qd Gs Bs area Vm Va baseKV zone Vmax Vmin")
bus_data = []
for i, row in buses.iterrows():
    bus_type = 1  # PQ
    if row["control"].lower() == "pv":
        bus_type = 2
    elif row["control"].lower() == "slack":
        bus_type = 3
    pd = loads[loads["bus"] == row["name"]]["p_set"].sum() if row["name"] in loads["bus"].values else 0
    qd = loads[loads["bus"] == row["name"]]["q_set"].sum() if row["name"] in loads["bus"].values else 0
    bus_data.append(f"{i+1} {bus_type} {pd} {qd} 0 0 1 1 0 {row['v_nom']} 1 1.1 0.9")
matpower_content.append("\n".join(bus_data))

# Generator data
matpower_content.append("\n%% generator data")
matpower_content.append("% bus Pg Qg Qmax Qmin Vg mBase status Pmax Pmin Pc1 Pc2 Qc1min Qc1max Qc2min Qc2max ramp_agc ramp_10 ramp_30 ramp_q apf")
gen_data = []
for _, row in generators.iterrows():
    bus_id = bus_index[row["bus"]]
    pg = row["p_set"]
    pmax = row["p_nom"]
    qmax = pmax * 0.5  # Assume Qmax = Pmax/2 (adjust if needed)
    qmin = -qmax
    vg = 1.0  # Default voltage setpoint
    gen_data.append(f"{bus_id} {pg} 0 {qmax} {qmin} {vg} 100 1 {pmax} 0 0 0 0 0 0 0 0 0 0 0 0")
matpower_content.append("\n".join(gen_data))

# Branch data (lines and transformers)
matpower_content.append("\n%% branch data")
matpower_content.append("% fbus tbus r x b rateA rateB rateC ratio angle status angmin angmax")
branch_data = []
# Lines
for _, row in lines.iterrows():
    fbus = bus_index[row["bus0"]]
    tbus = bus_index[row["bus1"]]
    rate_a = 100  # Assume 100 MVA if not specified
    branch_data.append(f"{fbus} {tbus} {row['r']} {row['x']} {row['b']} {rate_a} {rate_a} {rate_a} 0 0 1 -360 360")
# Transformers
for _, row in transformers.iterrows():
    fbus = bus_index[row["bus0"]]
    tbus = bus_index[row["bus1"]]
    rate_a = row["s_nom"]
    branch_data.append(f"{fbus} {tbus} {row['r']} {row['x']} 0 {rate_a} {rate_a} {rate_a} 0 0 1 -360 360")
matpower_content.append("\n".join(branch_data))

# Generator cost (minimal for load flow)
matpower_content.append("\n%% generator cost data")
matpower_content.append("% 1 startup shutdown n x1 y1 ... xn yn")
matpower_content.append("% 2 startup shutdown n c(n-1) ... c0")
gencost_data = []
for _ in generators.iterrows():
    gencost_data.append("2 0 0 2 0 0")
matpower_content.append("\n".join(gencost_data))

# Combine MATPOWER content
matpower_content = "\n".join(matpower_content)


INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 5 iterations with error of 0.000000 in 0.177883 seconds


In [11]:
# Append to the .m file
with open("network.m", "a") as f:
    f.write(matpower_content + "\n")

In [12]:

import pypsa
import pandas as pd
import numpy as np

# Initialize PyPSA network
n = pypsa.Network()

# Load CSV files
buses = pd.read_csv("Nepal_Data/buses.csv")
loads = pd.read_csv("Nepal_Data/loads.csv")
generators = pd.read_csv("Nepal_Data/generators.csv")
lines = pd.read_csv("Nepal_Data/lines.csv")
transformers = pd.read_csv("Nepal_Data/transformers.csv")

# Create bus index mapping (1-based for MATPOWER)
bus_index = {row["name"]: i + 1 for i, row in buses.iterrows()}

# Add buses to PyPSA network
for _, row in buses.iterrows():
    n.add("Bus", row["name"], v_nom=row["v_nom"], type=row["control"].lower())

# Add loads
for _, row in loads.iterrows():
    n.add("Load", row["name"], bus=row["bus"], p_set=row["p_set"], q_set=row["q_set"])

# Add generators
for _, row in generators.iterrows():
    n.add("Generator", row["name"], bus=row["bus"], control=row["control"], p_nom=row["p_nom"], p_set=row["p_set"])

# Add lines
for _, row in lines.iterrows():
    n.add("Line", row["name"], bus0=row["bus0"], bus1=row["bus1"], r=row["r"], x=row["x"], b=row["b"], s_nom=100)

# Add transformers
for _, row in transformers.iterrows():
    n.add("Transformer", row["name"], bus0=row["bus0"], bus1=row["bus1"], r=row["r"], x=row["x"], s_nom=row["s_nom"])

# Run power flow to validate
n.pf()

# Create MATPOWER NumPy arrays
# mpc.bus: [bus_i, type, Pd, Qd, Gs, Bs, area, Vm, Va, baseKV, zone, Vmax, Vmin]
bus_array = np.zeros((len(buses), 13))
for i, row in buses.iterrows():
    bus_type = 1  # PQ
    if row["control"].lower() == "pv":
        bus_type = 2
    elif row["control"].lower() == "slack":
        bus_type = 3
    pd = loads[loads["bus"] == row["name"]]["p_set"].sum() if row["name"] in loads["bus"].values else 0
    qd = loads[loads["bus"] == row["name"]]["q_set"].sum() if row["name"] in loads["bus"].values else 0
    bus_array[i] = [i + 1, bus_type, pd, qd, 0, 0, 1, 1, 0, row["v_nom"], 1, 1.1, 0.9]

# mpc.gen: [bus, Pg, Qg, Qmax, Qmin, Vg, mBase, status, Pmax, Pmin, Pc1, Pc2, Qc1min, Qc1max, Qc2min, Qc2max, ramp_agc, ramp_10, ramp_30, ramp_q, apf]
gen_array = np.zeros((len(generators), 21))
for i, row in generators.iterrows():
    bus_id = bus_index[row["bus"]]
    pg = row["p_set"]
    pmax = row["p_nom"]
    qmax = pmax * 0.5  # Assume Qmax = Pmax/2
    qmin = -qmax
    gen_array[i] = [bus_id, pg, 0, qmax, qmin, 1.0, 100, 1, pmax, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# mpc.branch: [fbus, tbus, r, x, b, rateA, rateB, rateC, ratio, angle, status, angmin, angmax]
branch_array = np.zeros((len(lines) + len(transformers), 13))
# Lines
for i, row in lines.iterrows():
    fbus = bus_index[row["bus0"]]
    tbus = bus_index[row["bus1"]]
    rate_a = 100  # Assume 100 MVA
    branch_array[i] = [fbus, tbus, row["r"], row["x"], row["b"], rate_a, rate_a, rate_a, 0, 0, 1, -360, 360]
# Transformers
for i, row in transformers.iterrows():
    fbus = bus_index[row["bus0"]]
    tbus = bus_index[row["bus1"]]
    rate_a = row["s_nom"]
    branch_array[i + len(lines)] = [fbus, tbus, row["r"], row["x"], 0, rate_a, rate_a, rate_a, 0, 0, 1, -360, 360]

# mpc.gencost: [model, startup, shutdown, n, c(n-1), ..., c0]
gencost_array = np.array([[2, 0, 0, 2, 0, 0] for _ in range(len(generators))])

# Print shapes to verify
print(f"mpc.bus shape: {bus_array.shape}")
print(f"mpc.gen shape: {gen_array.shape}")
print(f"mpc.branch shape: {branch_array.shape}")
print(f"mpc.gencost shape: {gencost_array.shape}")

# Optionally, save arrays to a .npz file
np.savez("matpower_data.npz", bus=bus_array, gen=gen_array, branch=branch_array, gencost=gencost_array)

# Return arrays for inspection
{
    "mpc.bus": bus_array.tolist(),
    "mpc.gen": gen_array.tolist(),
    "mpc.branch": branch_array.tolist(),
    "mpc.gencost": gencost_array.tolist()
}


INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 5 iterations with error of 0.000000 in 0.094192 seconds


mpc.bus shape: (223, 13)
mpc.gen shape: (46, 21)
mpc.branch shape: (270, 13)
mpc.gencost shape: (46, 6)


{'mpc.bus': [[1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   1.0,
   0.0,
   132.0,
   1.0,
   1.1,
   0.9],
  [2.0, 1.0, 14.11, 8.744592615, 0.0, 0.0, 1.0, 1.0, 0.0, 33.0, 1.0, 1.1, 0.9],
  [3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 132.0, 1.0, 1.1, 0.9],
  [4.0, 1.0, 13.77, 8.53387954, 0.0, 0.0, 1.0, 1.0, 0.0, 33.0, 1.0, 1.1, 0.9],
  [5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 11.0, 1.0, 1.1, 0.9],
  [6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 132.0, 1.0, 1.1, 0.9],
  [7.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 132.0, 1.0, 1.1, 0.9],
  [8.0, 1.0, 6.8, 4.214261501, 0.0, 0.0, 1.0, 1.0, 0.0, 33.0, 1.0, 1.1, 0.9],
  [9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 132.0, 1.0, 1.1, 0.9],
  [10.0, 1.0, 6.8, 4.214261501, 0.0, 0.0, 1.0, 1.0, 0.0, 33.0, 1.0, 1.1, 0.9],
  [11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 132.0, 1.0, 1.1, 0.9],
  [12.0,
   1.0,
   14.11,
   8.744592615,
   0.0,
   0.0,
   1.0,
   1.0,
   0.0,
   33.0,
   1.0,
   1.1,
   0.9],
  [13.0, 1.0, 0.0, 0.0,